# EQTransformer 1 Day Test

### Using Raspberry Shake earthquake data from 12-29-2022 recorded within a 5° circle around the islands of Hispañola and Puerto Rico.

##### Data was put through the BlocklyEQTransformer machine learning tool to pick out predicted P and S waves. This is set up here along with a  theoretical catalog of events from USGS in order to compare P and S wave arrival times, determining which events were picked up by the machine learning tool versus USGS and vice versa.


In [26]:
from obspy.core.inventory.inventory import read_inventory
from obspy.taup import TauPyModel
from obspy.core.event import read_events
from obspy import read
from obspy.geodetics import  kilometer2degrees as km2deg
import matplotlib.pyplot as plt
import numpy as np
import os
from obspy.taup import TauPyModel
from obspy.taup import plot_travel_times
import pandas as pd 
from obspy.geodetics.base import gps2dist_azimuth as ll2daz
from obspy import UTCDateTime as UTC
import glob
model = TauPyModel(model="iasp91")

In [27]:
#creating events catalog

events = pd.read_csv('https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2022-12-29T00:00:00&endtime=2022-12-30T00:00:00&latitude=17.5&longitude=-70&maxradius=5')

#creating list of stations

inv = pd.read_json('station_list.json').transpose()
inv['stat'] = inv.index
stationslist = []
for index, row in inv.iterrows():
    [lat,lon,depth] = row['coords']
    listadd = (lat,lon,depth,row['stat'])
    stationslist.append(listadd)


In [28]:
#copying stationlist
import shutil
shutil.copy('/home/bmacbeth3/project/BlocklyEQTransformer/scripts/PR_20221229/knownevents/station_list.json', '/home/bmacbeth3/project/BlocklyEQTransformer/scripts/PR_20221229/knownevents/RSML' )

'/home/bmacbeth3/project/BlocklyEQTransformer/scripts/PR_20221229/knownevents/RSML/station_list.json'

In [29]:
#calculating list of THEORETICAL arrival times for P and S waves given regional earthquake locations and origin times
#makes use of Obspy's implementation of the TauP algorithm

phaselist=[]
for station in range(len(stationslist)):
    slat = stationslist[station][0]
    slon = stationslist[station][1]
    stat = stationslist[station][3]
    for each in range(len(events)):
        elat = events.latitude[each]
        elon = events.longitude[each] 
        edep = events.depth[each]
        distazbaz = ll2daz(slat, slon, elat, elon)
        distdeg = km2deg(distazbaz[0]/1000.)
        Parrivals = model.get_travel_times(source_depth_in_km=edep,
                                   distance_in_degree = distdeg,
                                   phase_list=["P","p", "Pn","Pg"])
        Sarrivals = model.get_travel_times(source_depth_in_km=edep,
                                   distance_in_degree = distdeg,
                                   phase_list=["S","s","Sn","Sg"])
        Ps = []
        Ss = []
        for i in range(len(Parrivals)):
            Ps.append(UTC(events.time[each])+Parrivals[i].time)
        for i in range(len(Sarrivals)):
            Ss.append(UTC(events.time[each])+Sarrivals[i].time)
        
        phaselist.append([[stat,slat,slon],[elat,elon,edep,UTC(events.time[each])],distdeg,Ps,Ss])

#creating a dataframe for the THEORETICAL arrival times

phasedata = pd.DataFrame(phaselist, columns = ["station", "origin", "distdeg", "Ps", "Ss"])
#phasedata.tail()

In [30]:
#creating dataframe of PREDICTED arrival times which were calculated with the BlocklyEQTransformer machine learning tool

DetectDf = pd.DataFrame()

detectdirbase = '/home/bmacbeth3/project/BlocklyEQTransformer/scripts/PR_20221229/detections/'
for path in glob.iglob(detectdirbase + "*_outputs" ): #+ "X_prediction_results.csv"):
    path=str(path+"/X_prediction_results.csv")
    dflocal = pd.read_csv(path, ",")
    DetectDf = DetectDf.append(dflocal, ignore_index = True)
    
DetectDf

/home/bmacbeth3/anaconda3/envs/bet/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,file_name,network,station,instrument_type,station_lat,station_lon,station_elv,event_start_time,event_end_time,detection_probability,detection_uncertainty,p_arrival_time,p_probability,p_uncertainty,p_snr,s_arrival_time,s_probability,s_uncertainty,s_snr
0,R16A2/AM.R16A2.00.EHZ__20221229T000000Z__20221...,AM,R16A2,EH,18.243243,-74.086575,73.0,2022-12-29 22:44:59.352000,2022-12-29 22:45:11.332000,0.62,NaN,NaN,NaN,NaN,NaN,2022-12-29 22:45:08.102000,0.16,NaN,4.1
1,R16A2/AM.R16A2.00.EHZ__20221229T000000Z__20221...,AM,R16A2,EH,18.243243,-74.086575,73.0,2022-12-29 23:14:43.042000,2022-12-29 23:14:51.552000,0.95,NaN,2022-12-29 23:14:43.102000,0.79,NaN,19.0,2022-12-29 23:14:49.192000,0.26,NaN,4.3
2,R2ABA/AM.R2ABA.00.EHZ__20221229T000000Z__20221...,AM,R2ABA,EH,18.504504,-72.287422,574.0,2022-12-29 18:05:15.964000,2022-12-29 18:05:23.464000,0.46,NaN,2022-12-29 18:05:15.824000,0.47,NaN,16.6,NaN,NaN,NaN,NaN
3,R2ABA/AM.R2ABA.00.EHZ__20221229T000000Z__20221...,AM,R2ABA,EH,18.504504,-72.287422,574.0,2022-12-29 18:05:50.814000,2022-12-29 18:05:59.084000,0.54,NaN,2022-12-29 18:05:51.064000,0.12,NaN,16.6,NaN,NaN,NaN,NaN
4,R2ABA/AM.R2ABA.00.EHZ__20221229T000000Z__20221...,AM,R2ABA,EH,18.504504,-72.287422,574.0,2022-12-29 18:06:35.144000,2022-12-29 18:06:43.264000,0.39,NaN,2022-12-29 18:06:35.114000,0.45,NaN,15.1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,R4802/AM.R4802.00.EHZ__20221229T000000Z__20221...,AM,R4802,EH,18.441441,-66.002581,35.0,2022-12-29 10:19:55.287000,2022-12-29 10:20:03.357000,0.63,NaN,2022-12-29 10:19:55.317000,0.28,NaN,7.4,2022-12-29 10:20:01.167000,0.17,NaN,11.0
172,R4802/AM.R4802.00.EHZ__20221229T000000Z__20221...,AM,R4802,EH,18.441441,-66.002581,35.0,2022-12-29 13:47:29.527000,2022-12-29 13:47:58.427000,0.64,NaN,2022-12-29 13:47:29.527000,0.60,NaN,5.1,NaN,NaN,NaN,NaN
173,R4802/AM.R4802.00.EHZ__20221229T000000Z__20221...,AM,R4802,EH,18.441441,-66.002581,35.0,2022-12-29 21:33:37.697000,2022-12-29 21:33:38.107000,0.31,NaN,NaN,NaN,NaN,NaN,2022-12-29 21:33:38.087000,0.11,NaN,10.7
174,RD269/AM.RD269.00.EHZ__20221229T000000Z__20221...,AM,RD269,EH,19.774775,-72.226377,51.0,2022-12-29 04:20:26.646000,2022-12-29 04:21:00.666000,0.88,NaN,2022-12-29 04:20:27.096000,0.12,NaN,0.1,2022-12-29 04:20:50.816000,0.15,NaN,1.7


In [25]:
#saving dataframe locally for others to use
DetectDf.to_pickle("DetectDf.pkl")

In [44]:
phasedata

,station,origin,distdeg,Ps,Ss
0,"[S4051, 18.30630631, -66.07594445]","[18.4691, -68.729, 121.0, 2022-12-29T22:12:33....",2.526439,[2022-12-29T22:13:14.133218Z],[2022-12-29T22:13:45.454833Z]
1,"[S4051, 18.30630631, -66.07594445]","[18.0156666666667, -65.3868333333333, 18.66, 2...",0.716710,"[2022-12-29T21:12:58.562695Z, 2022-12-29T21:12...","[2022-12-29T21:13:08.692318Z, 2022-12-29T21:13..."
2,"[S4051, 18.30630631, -66.07594445]","[17.9801666666667, -66.9448333333333, 9.73, 20...",0.888323,"[2022-12-29T18:34:55.429845Z, 2022-12-29T18:34...","[2022-12-29T18:35:07.847570Z, 2022-12-29T18:35..."
3,"[S4051, 18.30630631, -66.07594445]","[17.983, -66.9703333333333, 6.61, 2022-12-29T1...",0.909947,"[2022-12-29T16:31:29.343067Z, 2022-12-29T16:31...","[2022-12-29T16:31:42.031842Z, 2022-12-29T16:31..."
4,"[S4051, 18.30630631, -66.07594445]","[18.0033333333333, -66.8448333333333, 13.37, 2...",0.791381,"[2022-12-29T14:14:07.660330Z, 2022-12-29T14:14...","[2022-12-29T14:14:18.792946Z, 2022-12-29T14:14..."
...,...,...,...,...,...
535,"[R50D4, 18.22522523, -73.61282146]","[17.9615, -66.9686666666667, 13.16, 2022-12-29...",6.329826,"[2022-12-29T03:20:49.422993Z, 2022-12-29T03:20...","[2022-12-29T03:22:02.587244Z, 2022-12-29T03:22..."
536,"[R50D4, 18.22522523, -73.61282146]","[17.9388333333333, -66.9081666666667, 11.51, 2...",6.388734,"[2022-12-29T03:17:30.950108Z, 2022-12-29T03:17...","[2022-12-29T03:18:44.887090Z, 2022-12-29T03:18..."
537,"[R50D4, 18.22522523, -73.61282146]","[17.95, -66.9675, 10.56, 2022-12-29T02:01:00.8...",6.331626,"[2022-12-29T02:02:34.159868Z, 2022-12-29T02:02...","[2022-12-29T02:03:47.546178Z, 2022-12-29T02:03..."
538,"[R50D4, 18.22522523, -73.61282146]","[18.0535, -67.113, 20.44, 2022-12-29T01:46:48....",6.187757,"[2022-12-29T01:48:18.481131Z, 2022-12-29T01:48...","[2022-12-29T01:49:29.535139Z, 2022-12-29T01:49..."


In [58]:
theorPhase=phasedata.copy()
tbuf=30
# add column to theoretical detection DF
theorPhase["EQTdetect"]=np.nan
for idTher in theorPhase.index: # each row contains a unique theoretical prediction based on catalog seismicity
    PsTher=theorPhase["Ps"][idTher]
    stat=theorPhase["station"][idTher][0]
    for idDet in DetectDf[DetectDf["station"]==stat].index:
        if DetectDf["p_arrival_time"][idDet] is not np.nan:  # only when not NaN
            PPred=UTC(DetectDf["p_arrival_time"][idDet])
            if PPred >= UTC(min(PsTher)-tbuf) and PPred <= UTC(max(PsTher)+tbuf):
                theorPhase["EQTdetect"][idTher]=idDet 
                #fills the EQTdetect column with the index of the station in DetectDf
                #which has an event which fits within these parameters
        
                

/home/bmacbeth3/anaconda3/envs/bet/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [59]:
theorPhase[theorPhase["EQTdetect"]>0]

,station,origin,distdeg,Ps,Ss,EQTdetect
17,"[S4051, 18.30630631, -66.07594445]","[17.95, -66.9675, 10.56, 2022-12-29T02:01:00.8...",0.919661,"[2022-12-29T02:01:18.570347Z, 2022-12-29T02:01...","[2022-12-29T02:01:31.431460Z, 2022-12-29T02:01...",92.0
20,"[R4DB9, 18.01801802, -66.83860199]","[18.4691, -68.729, 121.0, 2022-12-29T22:12:33....",1.853180,[2022-12-29T22:13:05.662911Z],[2022-12-29T22:13:30.291215Z],89.0
22,"[R4DB9, 18.01801802, -66.83860199]","[17.9801666666667, -66.9448333333333, 9.73, 20...",0.107965,[2022-12-29T18:34:40.993094Z],[2022-12-29T18:34:42.927052Z],88.0
31,"[R4DB9, 18.01801802, -66.83860199]","[17.9813333333333, -66.8546666666667, 13.11, 2...",0.039591,[2022-12-29T05:47:50.504132Z],[2022-12-29T05:47:52.235465Z],79.0
34,"[R4DB9, 18.01801802, -66.83860199]","[18.2076666666667, -67.1275, 24.15, 2022-12-29...",0.333539,[2022-12-29T03:29:15.329029Z],[2022-12-29T03:29:20.757035Z],78.0
160,"[RE001, 18.41441441, -68.93603553]","[18.4691, -68.729, 121.0, 2022-12-29T22:12:33....",0.204083,[2022-12-29T22:12:50.711973Z],[2022-12-29T22:13:03.608007Z],5.0
180,"[R2974, 18.12612613, -67.10740855]","[18.4691, -68.729, 121.0, 2022-12-29T22:12:33....",1.579171,[2022-12-29T22:13:02.380487Z],[2022-12-29T22:13:24.420972Z],103.0
182,"[R2974, 18.12612613, -67.10740855]","[17.9801666666667, -66.9448333333333, 9.73, 20...",0.212295,[2022-12-29T18:34:42.729318Z],[2022-12-29T18:34:45.924063Z],102.0
194,"[R2974, 18.12612613, -67.10740855]","[18.2076666666667, -67.1275, 24.15, 2022-12-29...",0.083386,[2022-12-29T03:29:12.245907Z],[2022-12-29T03:29:15.428608Z],101.0
202,"[R34A9, 18.45945946, -66.34149666]","[17.9801666666667, -66.9448333333333, 9.73, 20...",0.746317,"[2022-12-29T18:34:52.725140Z, 2022-12-29T18:34...","[2022-12-29T18:35:03.178781Z, 2022-12-29T18:35...",13.0


In [ ]:
theorPhase=phasedata.copy()
tbuf=30
# add column to theotical detection DF
theorPhase["EQTdetect"]=np.nan
for idTher in theorPhase.index: # each row contains a unique theoretical prediction based on catalog seismicity
    PsTher=theorPhase["Ps"][idTher]
    #print(PsTher)
    stat=theorPhase["station"][idTher][0]
    #print(stat)
    for idDet in DetectDf[DetectDf["station"]==stat].index:
        if DetectDf["p_arrival_time"][idDet] is not np.nan:  # only when not NaN
            PPred=UTC(DetectDf["p_arrival_time"][idDet])
            if PPred >= UTC(min(PsTher)-tbuf) and PPred <= UTC(max(PsTher)+tbuf):
                #if PPred <= max(PsTher)+tbuf:
                theorPhase["EQTdetect"][idTher]=idDet
                #print(min(PsTher)-tbuf,max(PsTher)+tbuf,PPred)
        

In [109]:
#creating lists of PREDICTED and THEORETICAL events for looping over

PREDICTED = []
for index, row in DetectDf.iterrows():
    PREDICTED.append((row["station"], UTC(row["event_start_time"])))

THEORETICAL = []
for index, row in phasedata.iterrows():
    THEORETICAL.append((row["station"][0], min(row["Ps"])))

In [111]:
#looping through the PREDICTED and THEORETICAL lists in order to check for detected/undetected events
#seeing if the PREDICTED event station == the THEORETICAL event station AND if the times are within 30 seconds of each other

detected = 0
undetected = 0
for Tevent in THEORETICAL:
    Tstation = Tevent[0]
    Ttime = Tevent[1]
    for Pevent in PREDICTED:
        Pstation = Pevent[0]
        Ptime = Pevent[1]
        if Pstation == Tstation and Ptime > (Ttime - 30) and Ptime < (Ttime + 30):
            detected += 1
        else:
            undetected += 1
print("The number of detected events is", detected)
print("The number of undetected events is", undetected)

The number of detected events is 26
The number of undetected events is 95014


In [113]:
len(THEORETICAL)

540